In [1]:
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import RMSprop
# from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pickle
from pathlib import Path

Using TensorFlow backend.


# Retrieving saved data 

In [2]:
with open('./xtrain.npy','rb') as f1:
    xtrain = pickle.load(f1)
with open('./ytrain.npy','rb') as f2:
    ytrain = pickle.load(f2)

In [3]:
print(xtrain.shape)
print(ytrain.shape)

(21865, 5, 4)
(21865, 4)


In [6]:
notes = []

time = 0.0
prev_time = 0.0


music_path = Path('./pop-music-collection/Pop_Music_Midi/')

for fil in music_path.glob('*.midi'):  #Iterating over all midi files
    mid = MidiFile(fil)  #Reading midi file
#     print(fil)
    for msg in mid:
        time += msg.time
        if msg.type == 'note_on':  
        # including in training data only if recieved message is of type note_on
            note = msg.bytes() 
        # note message is in the form of [type, note, velocity]
        # taking only note and velocity
            note = note[1:]
            note.append(time-prev_time) 
            note.append(msg.channel)
            prev_time = time
            notes.append(note)
print(len(notes))
print(time)

mean = np.mean(notes,axis=0)
print(mean)
std = np.std(notes,axis=0)
print(std)
n_prev=5

21870
2744.0826523281203
[60.02926383 72.42930956  0.12544995  1.90964792]
[ 8.61605267 25.49014909  0.21616718  0.66362352]


# Model

In [7]:
model = Sequential()
model.add(LSTM(64, input_shape=(n_prev, 4), return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(32,return_sequences=False))
model.add(Dropout(0.25))
model.add(Dense(4))
model.add(Activation('linear'))

model.compile(loss='mse', optimizer='rmsprop',metrics = ['accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 5, 64)             17664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 64)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 132       
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0         
Total params: 30,212
Trainable params: 30,212
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(xtrain, ytrain, batch_size=50, epochs=20, verbose=1)

Epoch 1/20
21865/21865 [==============================] - 8s 383us/step - loss: 0.7271 - acc: 0.4321
Epoch 2/20
21865/21865 [==============================] - 6s 296us/step - loss: 0.5192 - acc: 0.5072
Epoch 3/20
21865/21865 [==============================] - 6s 296us/step - loss: 0.4767 - acc: 0.5353
Epoch 4/20
21865/21865 [==============================] - 7s 298us/step - loss: 0.4572 - acc: 0.5555
Epoch 5/20
21865/21865 [==============================] - 7s 302us/step - loss: 0.4424 - acc: 0.5710
Epoch 6/20
21865/21865 [==============================] - 7s 302us/step - loss: 0.4342 - acc: 0.5809
Epoch 7/20
21865/21865 [==============================] - 7s 304us/step - loss: 0.4257 - acc: 0.5865
Epoch 8/20
21865/21865 [==============================] - 7s 320us/step - loss: 0.4192 - acc: 0.5930
Epoch 9/20
21865/21865 [==============================] - 6s 282us/step - loss: 0.4155 - acc: 0.6013
Epoch 10/20
21865/21865 [==============================] - 8s 351us/step - loss: 0.4101 - a

# Generating Random Seed

In [9]:
temp = np.random.randint(0,xtrain.shape[0])
seed = xtrain[temp]
print(seed)

[[-3.39643139e-03 -1.62530668e+00 -5.80337635e-01  1.36149606e-01]
 [ 8.09040571e-01 -1.62530668e+00 -5.80337635e-01  1.36149606e-01]
 [ 4.60853284e-01 -1.62530668e+00 -5.80337635e-01  1.36149606e-01]
 [ 2.78210186e+00  1.16008307e+00  5.76174629e-01 -1.37072888e+00]
 [ 2.20178972e+00  1.16008307e+00 -2.08150311e-03 -1.37072888e+00]]


In [11]:
prediction = []
x = seed # x.shape = (4,5)
x = np.expand_dims(x, axis=0) # x.shape = (1,4,5)
for i in range(600):
    preds = model.predict(x)
    x = np.squeeze(x) # x.shape = (4,5)
    x = np.concatenate((x, preds))
    x = x[1:]
    x = np.expand_dims(x, axis=0) # x.shape = (1,4,5)
    preds = np.squeeze(preds) # preds.shape = (5,)
    prediction.append(preds)

ix=-1
for pred in prediction:
    ix+=1
    pred = (pred*std)+mean
    pred[0] = int(pred[0])
    pred[1] = int(pred[1])
    pred[3] = int(round(pred[3]))
    # rejecting unwanted values  
    if pred[0] < 24:
        pred[0] = 24
    elif pred[0] > 102:
        pred[0] = 102
    if pred[1] < 0:
        pred[1] = 0
    elif pred[1] > 127:
        pred[1] = 127
    if pred[2] < 0:
        pred[2] = 0
    prediction[ix] = pred
prediction_temp = np.array(prediction)
print(np.max(prediction_temp,axis=0))
print(np.min(prediction_temp,axis=0))

[75.        95.         0.4012941  3.       ]
[47. 35.  0.  1.]


In [12]:
print(prediction_temp)

[[6.00000000e+01 7.00000000e+01 1.94047056e-02 2.00000000e+00]
 [6.40000000e+01 7.70000000e+01 1.93709480e-02 2.00000000e+00]
 [5.90000000e+01 7.80000000e+01 0.00000000e+00 2.00000000e+00]
 ...
 [5.90000000e+01 7.70000000e+01 3.87891626e-01 2.00000000e+00]
 [6.30000000e+01 3.90000000e+01 2.43390958e-02 2.00000000e+00]
 [6.30000000e+01 3.50000000e+01 9.92638215e-04 2.00000000e+00]]


In [17]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

for note in prediction:
    channel = note[3]
    time = note[3]/0.0125 # to rescale to midi's delta ticks. arbitrary value for now.
    note = note[:2]
    # 147 means note_on
    note = np.insert(note, 0, 147)
    bytes = note.astype(int)
  
    msg = Message.from_bytes(bytes[0:3]) 
    
    msg.time = int(time)
    msg.channel = int(channel)
    track.append(msg)
    print (note)

mid.save('new_song.midi')

[147.  60.  70.]
[147.  64.  77.]
[147.  59.  78.]
[147.  50.  83.]
[147.  75.  95.]
[147.  69.  87.]
[147.  64.  62.]
[147.  63.  52.]
[147.  60.  48.]
[147.  64.  83.]
[147.  60.  62.]
[147.  62.  55.]
[147.  56.  55.]
[147.  55.  76.]
[147.  63.  46.]
[147.  62.  39.]
[147.  54.  48.]
[147.  55.  77.]
[147.  64.  47.]
[147.  63.  45.]
[147.  52.  50.]
[147.  56.  76.]
[147.  64.  45.]
[147.  63.  42.]
[147.  54.  48.]
[147.  53.  77.]
[147.  63.  54.]
[147.  62.  53.]
[147.  50.  56.]
[147.  58.  76.]
[147.  63.  40.]
[147.  63.  37.]
[147.  56.  44.]
[147.  50.  77.]
[147.  62.  70.]
[147.  61.  74.]
[147.  47.  74.]
[147.  61.  79.]
[147.  62.  45.]
[147.  64.  37.]
[147.  59.  39.]
[147.  52.  76.]
[147.  61.  68.]
[147.  61.  72.]
[147.  48.  74.]
[147.  60.  79.]
[147.  63.  45.]
[147.  63.  37.]
[147.  58.  41.]
[147.  52.  77.]
[147.  62.  66.]
[147.  61.  70.]
[147.  47.  71.]
[147.  60.  79.]
[147.  62.  41.]
[147.  64.  35.]
[147.  59.  40.]
[147.  51.  78.]
[147.  61.  70